# Compiling a QGL2 RabiAmp and plotting the output

In [ ]:
# Using phase=0, amp from 0 -> 1 stepping by 0.1

## Imports

In [ ]:
from pyqgl2.main import compile_function, compile_to_hardware
import QGL
from QGL.Plotting import output_notebook
from QGL.PulseSequencePlotter import plot_pulse_files
from QGL import QubitFactory
from pyqgl2.qreg import QRegister
import numpy as np
import os

In [ ]:
seqpath = "../src/python/qgl2/basic_sequences"

In [ ]:
output_notebook()

## Compile to QGL1

In [ ]:
# Insert proper path to QGL2 source and name of qgl2main if not so marked
# Here we compile the named function in the named file from QGL2 to QGL1 and return the new function
# intermediate_output option saves to the specified file

Uncomment below for debugging

In [ ]:
# from pyqgl2.ast_util import NodeError
# from pyqgl2.debugmsg import DebugMsg

# DebugMsg.set_level(1)
# NodeError.MUTE_ERR_LEVEL = NodeError.NODE_ERROR_NONE

In [ ]:
qR = QRegister('q1','q2')
amps = np.linspace(0,1,11)

## Compile function from file

In [ ]:
seqs = compile_function(os.path.join(seqpath, "RabiMin.py"), "doRabiAmp", (qR, amps, 0), intermediate_output="debug.txt")()

## Compile to machine instructions

In [ ]:
fileNames = compile_to_hardware(seqs, "Rabi/Rabi")

## Alternative: Write to file and compile
Define sequences on the fly in this notebook. First, a file needs to be created with the required imports. 

In [ ]:
if not os.path.exists('../user_seqs'):
    os.mkdir('../user_seqs')

In [ ]:
%%writefile  ../user_seqs/my_seqs.py
from qgl2.qgl2 import qgl2decl, qreg, QRegister
from qgl2.qgl1 import Utheta, MEAS, X, Id
from qgl2.util import init
import qgl2.basic_sequences.pulses

In [ ]:
%%writefile  -a ../user_seqs/my_seqs.py


@qgl2decl
def test(q:qreg, amps, phase):
    for amp in amps:
        init(q)
        Id(q, amp=amp, phase=phase)
        MEAS(q)

In [ ]:
seqs = compile_function("../user_seqs/my_seqs.py", "test", (qR, amps, 0))()

## Plot the sequences

In [ ]:
plot_pulse_files(fileNames)